In [0]:
import pandas as pd

# Read the CSV
loan_book = pd.read_csv("/Workspace/Users/rohankumarlnu@gmail.com/loan_book.csv")  # works in notebooks only
loan_book

# Convert to Spark DataFrame
spark_loan_book = spark.createDataFrame(loan_book)

# Register as a temporary SQL table

spark_loan_book.createOrReplaceTempView("loan_book")


In [0]:
loan_level_panel= pd.read_csv("/Workspace/Users/rohankumarlnu@gmail.com/loan_level_panel.csv")
loan_level_panel

spark_loan_level_panel= spark.createDataFrame(loan_level_panel)
spark_loan_level_panel.createOrReplaceTempView("loan_level_panel")



In [0]:
macro= pd.read_csv("/Workspace/Users/rohankumarlnu@gmail.com/macro.csv")
macro

spark_macro= spark.createDataFrame(macro)
spark_macro.createOrReplaceTempView("macro")


In [0]:
macro_scenario_inputs= pd.read_csv("/Workspace/Users/rohankumarlnu@gmail.com/macro_scenario_inputs.csv")
macro_scenario_inputs

spark_macro_scenario_inputs= spark.createDataFrame(macro_scenario_inputs)
spark_macro_scenario_inputs.createOrReplaceTempView("macro_scenario_inputs")

In [0]:
stress_position = pd.read_csv("/Workspace/Users/rohankumarlnu@gmail.com/stress_position.csv")
stress_position

spark_stress_position= spark.createDataFrame(stress_position)
spark_stress_position.createOrReplaceTempView("stress_position")

In [0]:
trading_book= pd.read_csv("/Workspace/Users/rohankumarlnu@gmail.com/trading_book.csv")
trading_book

spark_trading_book= spark.createDataFrame(trading_book)
spark_trading_book.createOrReplaceTempView("trading_book")

In [0]:
isg_lending= pd.read_csv("/Workspace/Users/rohankumarlnu@gmail.com/isg_lending.csv")
isg_lending

spark_isg_lending= spark.createDataFrame(isg_lending)
spark_isg_lending.createOrReplaceTempView("isg_lending")

In [0]:
%sql

select * from loan_book

loan_id,orig_date,fico_bin,cltv_bin,MOB,product_type,exposure,as_of_date
L001,1/1/2022,660-700,80-90,39.0,Term Loan,"1,000",4/1/2025
L002,6/1/2023,720-750,70-80,22.0,Revolver,500,4/1/2025
L003,4/1/2021,600-640,90-100,48.0,Term Loan,"2,000",4/1/2025
null,null,null,null,null,null,null,null


In [0]:
%sql 
select * from trading_book


trade_id,instrument_type,issuer,market_value,sector,rating,duration,spread,as_of_date,Unnamed: 9
TB001,Bond,T-Mobile,3000000.0,Telecom,BBB,5.0,2.3,3/31/2025,null
TB002,CDS,Chevron,1500000.0,Energy,A,3.0,1.8,3/31/2025,null
TB003,MBS,FannieMae,2500000.0,Mortgages,AA,6.0,1.5,3/31/2025,null
null,null,null,null,null,null,null,null,null,null


In [0]:
%sql

select * from isg_lending



lending_id,counterparty,product_type,exposure_amt,loan_rating,origination_date,maturity_date,currency,sector,as_of_date
LND001,Apollo Capital,Term Loan,1.2E7,BBB,5/1/2021,5/1/2026,USD,Private Eq.,3/31/2025
LND002,GM Financial,Warehouse Line,8000000.0,BB+,8/15/2022,8/15/2027,USD,Auto,3/31/2025
LND003,Barclays,Revolving Credit,5000000.0,A,3/1/2023,3/1/2026,USD,Banking,3/31/2025
null,null,null,null,null,null,null,null,null,null


In [0]:
%sql

select product_type, sum(exposure_amt) as total_exposure from isg_lending group by product_type

product_type,total_exposure
Term Loan,1.2E7
Warehouse Line,8000000.0
Revolving Credit,5000000.0
null,null


In [0]:
%sql

select fico_bin, avg(default_rate) as avg_default from loan_level_panel group by fico_bin

fico_bin,avg_default
660-700,0.008
700-740,0.012
600-660,0.025
null,null


In [0]:
isg_exposures= pd.read_csv("/Workspace/Users/rohankumarlnu@gmail.com/isg_exposures.csv")
isg_exposures

spark_isg_exposures= spark.createDataFrame(isg_exposures)
spark_isg_exposures.createOrReplaceTempView("isg_exposures")

In [0]:
%sql
select * from isg_exposures

exposure_id,counterparty,sector,product_type,notional,rating,as_of_date
101,JPM,Financials,Bond,"25,000",A,3/31/2025
102,T-Mobile,Telecom,Loan,"10,000",BBB,3/31/2025
103,Blackstone,RealEstate,Repo,"5,000",A,3/31/2025
104,Shell,Energy,CDS,"8,000",AA,3/31/2025


In [0]:
%sql
create or replace temporary view isg_exposures_view as select *, cast(replace(notional, ',', '') as double)  as notional_double from isg_exposures

In [0]:
%sql

select * from isg_exposures_view

exposure_id,counterparty,sector,product_type,notional,rating,as_of_date,notional_double
101,JPM,Financials,Bond,"25,000",A,3/31/2025,25000.0
102,T-Mobile,Telecom,Loan,"10,000",BBB,3/31/2025,10000.0
103,Blackstone,RealEstate,Repo,"5,000",A,3/31/2025,5000.0
104,Shell,Energy,CDS,"8,000",AA,3/31/2025,8000.0


In [0]:

%sql

select sector, sum(notional_double) as total_exposure from isg_exposures_view group by sector;


sector,total_exposure
Financials,25000.0
Telecom,10000.0
RealEstate,5000.0
Energy,8000.0


In [0]:
%sql
create or replace temporary view stress_position_view as select * , cast(replace(exposure, ',', '') as double) as exposure_double from stress_position

In [0]:
%sql
select * from stress_position_view

segment_id,product_type,exposure,expected_loss,exposure_double
S001,Bond,"10,000",0.015,10000.0
S002,Loan,"5,000",0.03,5000.0
S003,Repo,"2,000",0.005,2000.0


In [0]:
%sql

select product_type , sum(exposure_double * expected_loss) as total_expected_loss from stress_position_view group by product_type

product_type,total_expected_loss
Bond,150.0
Loan,150.0
Repo,10.0


In [0]:
%sql
select * from loan_book

loan_id,orig_date,fico_bin,cltv_bin,MOB,product_type,exposure,as_of_date
L001,1/1/2022,660-700,80-90,39.0,Term Loan,"1,000",4/1/2025
L002,6/1/2023,720-750,70-80,22.0,Revolver,500,4/1/2025
L003,4/1/2021,600-640,90-100,48.0,Term Loan,"2,000",4/1/2025
null,null,null,null,null,null,null,null


In [0]:
%sql
create or replace temporary view loan_book_view as select * , cast(replace(exposure, ',', '') as double) as exposure_double from loan_book

In [0]:
%sql
select as_of_date, sum(exposure_double) as total_exposure from loan_book_view group by as_of_date order by as_of_date

as_of_date,total_exposure
null,null
4/1/2025,3500.0


In [0]:
%sql
select * from trading_book where as_of_date= '3/31/2025'

trade_id,instrument_type,issuer,market_value,sector,rating,duration,spread,as_of_date,Unnamed: 9
TB001,Bond,T-Mobile,3000000.0,Telecom,BBB,5.0,2.3,3/31/2025,null
TB002,CDS,Chevron,1500000.0,Energy,A,3.0,1.8,3/31/2025,null
TB003,MBS,FannieMae,2500000.0,Mortgages,AA,6.0,1.5,3/31/2025,null


In [0]:
bond_position= pd.read_csv("/Workspace/Users/rohankumarlnu@gmail.com/bond_positions.csv")
bond_position

,bond_id,issuer,market_price,prior_price,accrued_interest,as_of_date,Unnamed: 6,Unnamed: 7
0,B001,Apple,98.50,100.0,0.30,3/31/2025,NaN,NaN
1,B002,GM,95.25,99.0,0.45,3/31/2025,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
spark_bond_position= spark.createDataFrame(bond_position)
spark_bond_position.createOrReplaceTempView("bond_position")

In [0]:
%sql
select bond_id, market_price - prior_price as price_change, accrued_interest from bond_position

bond_id,price_change,accrued_interest
B001,-1.5,0.3
B002,-3.75,0.45
null,null,null
null,null,null


In [0]:
%sql
select as_of_date, sum(exposure_amt) as total_exposure from isg_lending group by as_of_date order by as_of_date

as_of_date,total_exposure
null,null
3/31/2025,2.5E7


In [0]:
%sql
select loan_id, count(*) from loan_book_view group by loan_id having count(*) >1

loan_id,count(*)
